In [1]:
import numpy as np
import math
import cv2
import mediapipe as mp
import pyautogui as pg
import keyboard

In [2]:
from pynput.mouse import Button, Controller

class RemoteMouse:
    def __init__(self):
        self.mouse = Controller()

    def getPosition(self):
        return self.mouse.position

    def setPos(self, xPos, yPos):
        self.mouse.position = (xPos, yPos)

    def movePos(self, xPos, yPos):
        self.mouse.move(xPos, yPos)

    def click(self):
        self.mouse.click(Button.left)

    def doubleClick(self):
        self.mouse.click(Button.left, 2)

    def clickRight(self):
        self.mouse.click(Button.right)

    def drag(self, from_x, from_y, to_x, to_y, is_absolute=True):
        if is_absolute is True:
            self.mouse.position = (from_x, from_y)
        else:
            self.mouse.position = self.getPosition()
            self.click()
            self.mouse.move(from_x, from_y)
        self.click()
        self.mouse.press(Button.left)

        if is_absolute is True:
            self.mouse.position = (to_x, to_y)
        else:
            self.mouse.move(to_x, to_y)
        self.mouse.release(Button.left)

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
def create_image(h, w, d):
    image = np.zeros((h, w,  d), np.uint8)
    color = tuple(reversed((0,0,0)))
    image[:] = color
    return image

In [5]:
def hshape(arr):
    r = arr[0]*16 + arr[1]*8 + arr[2]*4 + arr[3]*2 + arr[4]*1
    return r

In [8]:
mouse = RemoteMouse()
cap = cv2.VideoCapture(0)

SENSE = 29 #감도

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
 
    cus_bef = [-1,-1] #커서(손가락)의 이전(before) 위치
    cus_cur = [-1,-1] #커서(손가락)의 현재(current) 위치
    
    clicking = False
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
        
        height = image.shape[0]
        width = image.shape[1]
        depth = image.shape[2]
        
        dimage = create_image(height, width, depth) #바탕이 될 검은 이미지
        
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB) #BGR을 RGB로 변환
  
        results = hands.process(image) #손동작 인식
 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) #Opencv영상처리를 위해 다시 BGR로
 
        if results.multi_hand_landmarks: #result값이 정상인 경우에만 후속 작업
            for hls in results.multi_hand_landmarks:
                
                #손바닥의 가장 아랫점(사용하지 않았음)
                stdp = (hls.landmark[0].x * 100, hls.landmark[0].y * 100)
                
                #각 손가락의 끝점
                finger = [(hls.landmark[4].x * 100, hls.landmark[4].y * 100),
                          (hls.landmark[8].x * 100, hls.landmark[8].y * 100),
                          (hls.landmark[12].x * 100, hls.landmark[12].y * 100),
                          (hls.landmark[16].x * 100, hls.landmark[16].y * 100),
                          (hls.landmark[20].x * 100, hls.landmark[20].y * 100)]
                
                #각 손가락이 접혔는지를 판별(discriminate)할 기준점
                discr = [(hls.landmark[2].x * 100, hls.landmark[2].y * 100),
                          (hls.landmark[6].x * 100, hls.landmark[6].y * 100),
                          (hls.landmark[10].x * 100, hls.landmark[10].y * 100),
                          (hls.landmark[14].x * 100, hls.landmark[14].y * 100),
                          (hls.landmark[17].x * 100, hls.landmark[17].y * 100)]
                
                #손가락이 접혔는지를 나타내는 boolean 자료형의 리스트
                is_folded = [(finger[0][0] > discr[0][0]),
                             (finger[1][1] > discr[1][1]),
                             (finger[2][1] > discr[2][1]),
                             (finger[3][1] > discr[3][1]),
                             (finger[4][1] > discr[4][1])]
                
                hs = hshape(is_folded) #손모양(0~31까지의 정수)
                
                #이동(검지-중지)
                if hs == 19:
                    cus_cur = [(finger[1][0]+finger[2][0])/2, (finger[1][1]+finger[2][1])/2]
                    
                    if cus_bef == [-1,-1]: cus_bef = cus_cur
                    
                    moveX = SENSE*(cus_cur[0]-cus_bef[0])
                    moveY = SENSE*(cus_cur[1]-cus_bef[1])
                    mouse.movePos(moveX,moveY)
                    cus_bef = cus_cur
                    
                else : cus_bef = [-1,-1]
                
                #클릭(중지-약지)
                if hs == 6:
                    #연속 클릭 방지
                    if clicking : pass
                    else :
                        clicking = True
                        mouse.click()
                        
                else : clicking = False
                
            cv2.putText(
                dimage, text='stdp=(%d,%d) d : %d %d %d %d %d' % (stdp[0],stdp[1],is_folded[0],is_folded[1],is_folded[2],is_folded[3],is_folded[4]), org=(10, 30),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
                color=255, thickness=3)
        
            mp_drawing.draw_landmarks(dimage, hls, mp_hands.HAND_CONNECTIONS)

        cv2.imshow('NMBNM', dimage)
    
        if cv2.waitKey(1) == 27:
            break
 
cap.release()
cv2.destroyWindow('NMBNM')